In [156]:
import re
import pandas as pd
import numpy as np

In [157]:
files_dire = '../data/raw/'

In [158]:
df = pd.read_csv(f'{files_dire}/melbourne_past_listings.csv')
# merge the past listing with the geocode
# this should only left with properties that have been listed in 2021-2022
geo_df = pd.read_csv(f'{files_dire}/geo.csv').iloc[: , 1:]
df = df.merge(geo_df, on='address')
df.head()

,year,month,bed,bath,car,land_raw,type,address,suburb,code,rent_raw,url,loc_address,lat,lon
0,2021,January,3.0,1.0,1.0,NaN,Rental_residential,"1/31 DANDENONG ROAD EAST, FRANKSTON",Frankston,3199,$330,https://www.oldlistings.com.au/real-estate/VIC...,"Dandenong Road East, Frankston, Melbourne, Cit...",-38.129890,145.132153
1,2012,February,3.0,1.0,1.0,NaN,Rental_residential,"1/31 DANDENONG ROAD EAST, FRANKSTON",Frankston,3199,$320 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Dandenong Road East, Frankston, Melbourne, Cit...",-38.129890,145.132153
2,2012,January,3.0,1.0,1.0,NaN,Rental_residential,"1/31 DANDENONG ROAD EAST, FRANKSTON",Frankston,3199,$340 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Dandenong Road East, Frankston, Melbourne, Cit...",-38.129890,145.132153
3,2021,January,3.0,1.0,2.0,NaN,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,$395 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
4,2021,January,3.0,1.0,2.0,NaN,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,$395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494


In [159]:
# now review how many NaN exist in each column
df.count()

year           1444798
month          1444798
bed            1444783
bath           1444778
car            1334452
land_raw         84021
type           1181668
address        1444798
suburb         1444798
code           1444798
rent_raw       1444788
url            1444798
loc_address    1444798
lat            1444798
lon            1444798
dtype: int64

In [160]:
# we also remove the column of the size of the land as only very small number of rows have records
df = df.drop(['land_raw'], axis = 1)

In [161]:
# as the research goal is mainly residential properties (for people to live in)
# hence we filter out all properties that have zero bedroom or bathroom
temp = ['bed', 'bath', 'car']
df[temp] = df[temp].fillna(0)
df[temp] = df[temp].astype('int64')
df = df[~(df[temp] == 0).any(axis=1)]
# print out the number of entries left
len(df)

1334440

In [162]:
# from the previous output of NaN count, the types of property seems broken
df['type'].value_counts()

House                 472388
Unit/apmt             240217
Rental_residential    176164
Townhouse             109990
AvailableNow           59740
Unit                   17719
Apartment               5689
Available               3738
Villa                   2032
AvailableDate           1891
Studio                  1737
Flat                     835
Sales_residential         22
Rural                     10
ForSale                    5
Villa,House                3
Terrace                    3
Duplex                     1
Acreage/semi-rural         1
Name: type, dtype: int64

In [163]:
# some of the types has a really low count hence these are removed
temp = ['Duplex', 'Acreage/semi-rural', 'Terrace', 'Villa,House', 'ForSale', 'Rural', 'Sales_residential', 'AvailableDate', 'Villa', 'AvailableNow', 'Rental_residential', 'Available']

In [164]:
df = df[~df['type'].isin(temp)]
len(df)

1090830

In [165]:
# we want to combine categories in to three: house; Apartment / Unit / Flat; townhouse; studio
df['type'].value_counts()

House        472388
Unit/apmt    240217
Townhouse    109990
Unit          17719
Apartment      5689
Studio         1737
Flat            835
Name: type, dtype: int64

In [166]:
# 'AUF' stands for 'Apartment / Unit / Flat'
df['type'] = df['type'].replace(['Unit/apmt', 'Apartment', 'Flat', 'Unit'], 'AUF')
df['type'].value_counts()

House        472388
AUF          264460
Townhouse    109990
Studio         1737
Name: type, dtype: int64

In [167]:
df.head()

,year,month,bed,bath,car,type,address,suburb,code,rent_raw,url,loc_address,lat,lon
3,2021,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,$395 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
4,2021,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,$395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
5,2020,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,$395 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
6,2020,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,$395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
7,2017,December,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,$395 pw,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494


In [168]:
df['rent_raw'] = df['rent_raw'].str.replace(',', '')
df['rent_raw'] = df['rent_raw'].str.replace('$', '')
df.head()

/var/folders/y5/myhyydfd6h9dlwv1qmqtqhmr0000gn/T/ipykernel_12926/4226869503.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rent_raw'] = df['rent_raw'].str.replace('$', '')


,year,month,bed,bath,car,type,address,suburb,code,rent_raw,url,loc_address,lat,lon
3,2021,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
4,2021,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
5,2020,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395 per week,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
6,2020,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494
7,2017,December,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395 pw,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494


In [169]:
df['rent_raw'] = df['rent_raw'].str.lower()
df['rent_disc'] = df['rent_raw'].str.extract('(\D+)')
df['rent_raw'] = df['rent_raw'].str.extract('(\d+)')

In [170]:
df.head()

,year,month,bed,bath,car,type,address,suburb,code,rent_raw,url,loc_address,lat,lon,rent_disc
3,2021,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,per week
4,2021,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,NaN
5,2020,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,per week
6,2020,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,NaN
7,2017,December,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,395,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,pw


In [171]:
deduplicate_subset = ['year', 'month', 'bed', 'bath', 'car', 'type', 'address', 'suburb', 'code', 'rent_raw']

In [172]:
df = df.drop_duplicates(subset=deduplicate_subset, keep=False)
df = df[df.groupby('address').address.transform('count') > 1]

In [173]:
df['rent_disc'] = df['rent_disc'].str.replace(r'[^\w\s]+', '')
df['rent_disc'] = df['rent_disc'].str.replace(' ', '')

/var/folders/y5/myhyydfd6h9dlwv1qmqtqhmr0000gn/T/ipykernel_12926/3915710495.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['rent_disc'] = df['rent_disc'].str.replace(r'[^\w\s]+', '')


In [174]:
df['rent_disc'].value_counts()

perweek           177155
pw                106454
weekly             31928
                   23731
wk                  3354
                   ...  
forauction             1
priceonenquiry         1
millionweekly          1
justlisted             1
opentooffers           1
Name: rent_disc, Length: 84, dtype: int64

In [175]:
week = ['perweek', 'pw', 'weekly', 'wk', 'week', 'perweekgst']
month = ['permonth', 'monthly', 'pcm', 'pm', 'month']
year = ['pa', 'perannum', 'annually']

In [176]:
df['rent_disc'] = df['rent_disc'].replace(week, 7)
df['rent_disc'] = df['rent_disc'].replace(month, 30)
df['rent_disc'] = df['rent_disc'].replace(year, 365)

In [177]:
df.head()

,year,month,bed,bath,car,type,address,suburb,code,rent_raw,url,loc_address,lat,lon,rent_disc
9,2016,December,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,350,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,7
10,2016,December,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,330,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,7
11,2016,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,330,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,7
12,2015,November,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,330,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,7
13,2021,January,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,420,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,145.152331,7


In [197]:
allowed_vals = [7, 30, 365]
df = df[df['rent_disc'].isin(allowed_vals)]
df['rent_disc'] = df['rent_disc'].astype('int')
df = df[~df['rent_raw'].isnull()]
df = df[df['rent_raw'].str.isnumeric()]
df['rent_raw'] = df['rent_raw'].astype('int')

In [198]:
df.dtypes

year             int64
month           object
bed              int64
bath             int64
car              int64
type            object
address         object
suburb          object
code             int64
rent_raw         int64
url             object
loc_address     object
lat            float64
lon            float64
rent_disc        int64
dtype: object

In [203]:
df['weekly_rent'] =  df['rent_raw'] / df['rent_disc'] * 7
df = df.drop(['rent_disc', 'rent_raw'], axis=1)

In [204]:
df.head()

,year,month,bed,bath,car,type,address,suburb,code,url,loc_address,lat,lon,weekly_rent
9,2016,December,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,350.0
10,2016,December,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0
11,2016,January,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0
12,2015,November,3,1,2,House,"4 LUCERNE CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Lucerne Crescent, Frankston, Melbourne, City o...",-38.146375,145.166494,330.0
13,2021,January,3,1,3,House,"19 FRALLON CRESCENT, FRANKSTON",Frankston,3199,https://www.oldlistings.com.au/real-estate/VIC...,"Frallon Crescent, Karingal, Frankston, Melbour...",-38.136790,145.152331,420.0
